In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

In [1]:
import graphlab

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
sales['floors'] = sales['floors'].astype(int) 

This non-commercial license of GraphLab Create for academic use is assigned to ayushjain7398@gmail.com and will expire on June 23, 2019.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Ayushj\AppData\Local\Temp\graphlab_server_1530548589.log.0


In [3]:
import numpy as np # note this allows us to refer to numpy as np instead 

In [4]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 # this is how you add a constant column to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the others:
    features = ['constant'] + features # this is how you combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [5]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)

    return(predictions)

# Normalize features
In the house dataset, features vary wildly in their relative magnitude: `sqft_living` is very large overall compared to `bedrooms`, for instance. As a result, weight for `sqft_living` would be much smaller than weight for `bedrooms`. This is problematic because "small" weights are dropped first as `l1_penalty` goes up. 

To give equal considerations for all features, we need to **normalize features** as discussed in the lectures: we divide each feature by its 2-norm so that the transformed feature has norm 1.

Let's see how we can do this normalization easily with Numpy: let us first consider a small matrix.

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print X

[[  3.   5.   8.]
 [  4.  12.  15.]]


In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print norms

[  5.  13.  17.]


In [8]:
print X / norms # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[ 0.6         0.38461538  0.47058824]
 [ 0.8         0.92307692  0.88235294]]


In [9]:
def normalize_features(feature_matrix):
    norms = np.linalg.norm(feature_matrix, axis=0)
    normalized_features=feature_matrix / norms
    return(normalized_features, norms)
    

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print features
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print norms
# should print
# [5.  10.  15.]

[[ 0.6  0.6  0.6]
 [ 0.8  0.8  0.8]]
[  5.  10.  15.]


# Implementing Coordinate Descent with normalized features

## Effect of L1 penalty

In [11]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

In [12]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

In [13]:
weights = np.array([1., 4., 1.])

In [14]:
prediction =  predict_output(simple_feature_matrix,weights)
prediction

array([ 0.02675867,  0.04339256,  0.01990703, ...,  0.02289873,
        0.03178473,  0.02289873])

In [15]:
ro = [0 for i in range((simple_feature_matrix.shape)[1])]
for i in range((simple_feature_matrix.shape)[1]):
    ro[i] = (simple_feature_matrix[:,i] * (output - prediction + (weights[i] * simple_feature_matrix[:,i]))).sum()
print ro

[79400300.034929156, 87939470.772991076, 80966698.675965652]


In [16]:
# Return True if value is within the threshold ranges otherwise False
# Looking for range -l1_penalty/2 <= ro <= l1_penalty/2
def in_l1range(value, penalty):
    return ( (value >= -penalty/2.) and (value <= penalty/2.) )

In [17]:

for l1_penalty in [1.4e8, 1.64e8, 1.73e8, 1.9e8, 2.3e8]:
    print in_l1range(ro[1], l1_penalty), in_l1range(ro[2], l1_penalty)

False False
False True
False True
True True
True True


In [18]:
#1.9e8, 2.3e8

## Single Coordinate Descent Step

In [19]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    prediction = predict_output(feature_matrix,weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i =  (feature_matrix[:,i] * (output - prediction + (weights[i] * feature_matrix[:,i]))).sum()

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i+l1_penalty/2.
    elif ro_i > l1_penalty/2.:
        new_weight_i =ro_i-l1_penalty/2.
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [20]:
# should print 0.425558846691
import math
print lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1)

0.425558846691


## Cyclical coordinate descent 

In [34]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance):
    D=feature_matrix.shape[1]
    weights=np.array(initial_weights)
    converge=False
    change=np.array(initial_weights) * 0.0
    while not converge:
        for idx in range(D):
            new_weight = lasso_coordinate_descent_step(idx, feature_matrix,
                                                       output, weights,
                                                       l1_penalty)
            change[idx] = np.abs(new_weight - weights[idx])
            weights[idx] = new_weight
        max_change = max(change)
        if max_change < tolerance:
            converge = True
    return weights

In [35]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0

In [36]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

In [38]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance)
weights

array([ 21624998.36636292,  63157246.78545421,         0.        ])

In [46]:
pred2=predict_output(normalized_simple_feature_matrix,weights)
RSS = np.dot(output-pred2, output-pred2)
print RSS

1.63049248148e+15


# Evaluating LASSO fit with more features

In [47]:
train_data,test_data = sales.random_split(.8,seed=0)

In [48]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

In [52]:
(simple_matrix,output)=get_numpy_data(train_data,all_features,'price')
(normal_matrix,norms)=normalize_features(simple_matrix)

In [70]:
initial_weights = np.zeros(len(all_features) + 1)
l1_penalty = 1e7
tolerance = 1.0

In [71]:
weights1e7 = lasso_cyclical_coordinate_descent(normal_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
print weights1e7

[ 24429600.60933314         0.                 0.          48389174.35227978
         0.                 0.           3317511.16271982   7329961.9848964
         0.                 0.                 0.                 0.
         0.                 0.        ]


In [72]:
feature_list = ['constant'] + all_features
print feature_list

['constant', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated']


In [73]:
feature_weights1e7 = dict(zip(feature_list, weights1e7))
for k,v in feature_weights1e7.iteritems():
    if v != 0.0:
        print k, v

sqft_living 48389174.3523
waterfront 3317511.16272
constant 24429600.6093
view 7329961.9849


In [74]:
l1_penalty=1e8
tolerance = 1.0
weights1e8 = lasso_cyclical_coordinate_descent(normal_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
weights1e8

array([ 71114625.75280938,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ,         0.        ,
               0.        ,         0.        ])

In [75]:
feature_weights1e8 = dict(zip(feature_list, weights1e8))
for k,v in feature_weights1e8.iteritems():
    if v != 0.0:
        print k, v

constant 71114625.7528


In [76]:
l1_penalty=1e4
tolerance = 5e5
weights1e4 = lasso_cyclical_coordinate_descent(normal_matrix, output,
                                               initial_weights, l1_penalty, tolerance)
weights1e4

array([ 77779073.91265215, -22884012.25023361,  15348487.08089997,
        92166869.69883084,  -2139328.0824278 ,  -8818455.54409496,
         6494209.73310655,   7065162.05053197,   4119079.21006765,
        18436483.52618778, -14566678.54514349,  -5528348.75179429,
       -83591746.20730527,   2784276.46012858])

In [93]:
feature_weights1e4 = dict(zip(feature_list, weights1e4))
for k,v in feature_weights1e4.iteritems():
    if v != 0.0:
        print k, v

bathrooms 15348487.0809
sqft_above -14566678.5451
grade 18436483.5262
yr_renovated 2784276.46013
bedrooms -22884012.2502
sqft_living 92166869.6988
floors -8818455.54409
condition 4119079.21007
waterfront 6494209.73311
constant 77779073.9127
sqft_basement -5528348.75179
yr_built -83591746.2073
sqft_lot -2139328.08243
view 7065162.05053


## Rescaling learned weights

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [78]:
(simple_matrix,output)=get_numpy_data(train_data,all_features,'price')
(normal_matrix,norms)=normalize_features(simple_matrix)

In [79]:
w1e4=weights1e4/norms
w1e7=weights1e7/norms
w1e8=weights1e8/norms

print w1e7[3]

161.317456248


## Evaluating each of the learned models on the test data

In [82]:
(test_feature_matrix, test_output) = get_numpy_data(test_data, all_features, 'price')
#(normal_test_matrix,norms)=normalize_features(test_feature_matrix)

In [87]:
prediction =  predict_output(test_feature_matrix, w1e7)
RSS = np.dot(test_output-prediction, test_output-prediction)
print  RSS

2.75962079909e+14


In [90]:
prediction1 =  predict_output(test_feature_matrix, w1e8)
RSS1 = np.dot(test_output-prediction1, test_output-prediction1)
print  RSS1

5.37166150034e+14
